In [10]:
# --- LIBRERÍAS ---
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import numpy as np

In [11]:
# --- CONFIGURACIÓN DE PANDAS ---
pd.set_option('display.max_columns', None)

# --- CARGA DE DATOS ---
clients = pd.read_csv('../DATA/clients_202510280926.csv')
orders = pd.read_csv('../DATA/orders_202510280929.csv')
products = pd.read_csv('../DATA/products_202510280931.csv')

# --- VERIFICAR ---
print("Clientes:", clients.shape)
print("Órdenes:", orders.shape)
print("Productos:", products.shape)

# --- VISTAS RÁPIDAS ---
display(clients.head())
display(orders.head())
display(products.head())


Clientes: (2357, 15)
Órdenes: (82245, 10)
Productos: (1255, 6)


,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,citizenship,document_type,document_number,registration_date,last_seen
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,Uruguay,CI,2719583-8,2025-05-02 00:00:00.000,2025-06-19 00:00:00.000
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,Uruguay,CI,1445199-8,2023-02-24 00:00:00.000,2023-12-09 00:00:00.000
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,Uruguay,CI,8090293-5,2024-10-03 00:00:00.000,2025-05-03 00:00:00.000
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,Uruguay,CI,7022674-5,2025-01-13 00:00:00.000,2025-08-11 00:00:00.000
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,Uruguay,CI,5918715-9,2025-03-09 00:00:00.000,2025-03-21 00:00:00.000


,order_id,client_id,product_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost
0,1,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00333,Snacks,29.51,2023-01-01 23:51:55.000,delivered,bank_transfer,standard,3.51
1,2,0c8380cb-a64e-4090-8b23-d1ddf32630e0,P00929,Personal Care,23.40,2023-01-01 11:48:50.000,delivered,credit_card,standard,3.76
2,3,bf486527-8182-490e-9e10-c79232c6b0c9,P00768,Personal Care,94.41,2023-01-01 11:06:14.000,delivered,debit_card,express,10.42
3,5,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00610,Household,29.16,2023-01-01 20:55:35.000,delivered,credit_card,standard,6.05
4,6,7ac7a850-4f19-4c85-921b-b0e5ba921ee9,P00299,Snacks,76.22,2023-01-01 02:49:09.000,delivered,debit_card,express,14.08


,product_id,product_name,category,brand,last_updated,created_at
0,P00001,Lipton Ice Tea,Beverages,Unilever,2025-09-06 21:21:31.000,2025-04-06 21:21:31.000
1,P00002,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2025-05-02 21:21:31.000
2,P00003,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2024-10-02 21:21:31.000
3,P00004,Lipton Ice Tea,Beverages,Unilever,2025-09-06 21:21:31.000,2025-04-21 21:21:31.000
4,P00005,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2025-04-22 21:21:31.000


In [12]:
# --- VARIABLES EN CADA TABLA ---

print("🧍‍♂️ CLIENTS:")
print(clients.columns.tolist())
print("\n🛒 ORDERS:")
print(orders.columns.tolist())
print("\n📦 PRODUCTS:")
print(products.columns.tolist())


🧍‍♂️ CLIENTS:
['client_id', 'first_name', 'last_name', 'gender', 'age', 'email', 'city', 'address', 'postal_code', 'country', 'citizenship', 'document_type', 'document_number', 'registration_date', 'last_seen']

🛒 ORDERS:
['order_id', 'client_id', 'product_id', 'product_category', 'order_price', 'order_timestamp', 'order_status', 'payment_method', 'shipping_method', 'shipping_cost']

📦 PRODUCTS:
['product_id', 'product_name', 'category', 'brand', 'last_updated', 'created_at']


In [14]:
# # ---------- Configuración de fechas ----------
# CUTOFF = pd.Timestamp('2024-08-31')
# MAX_DATE = pd.Timestamp('2024-12-31')
# HORIZON = 90  # días

# # ---------- Asegurar tipos de fecha ----------
# orders['order_timestamp'] = pd.to_datetime(orders['order_timestamp'])
# # si no existe order_date, lo creamos
# if 'order_date' not in orders.columns:
#     orders['order_date'] = orders['order_timestamp'].dt.date
# orders['order_date'] = pd.to_datetime(orders['order_date'])

# products['last_updated'] = pd.to_datetime(products['last_updated'], errors='coerce')
# clients['last_seen'] = pd.to_datetime(clients['last_seen'], errors='coerce')

# # ---------- Recorte temporal (máximo 31/12/2024) ----------
# orders = orders[orders['order_timestamp'] <= MAX_DATE]
# products = products[products['last_updated'].isna() | (products['last_updated'] <= MAX_DATE)]

# # ---------- Unir orders con products para traer category / last_updated ----------
# ord_prod = orders.merge(
#     products[['product_id','category','last_updated']],
#     on='product_id', how='left'
import pandas as pd

# ---------- Configuración de fechas ----------
CUTOFF  = pd.Timestamp('2024-08-31')
MAX_DATE = pd.Timestamp('2024-12-31')
HORIZON = 90  # días

# ---------- Trabajar sobre copias ----------
_orders   = orders.copy()
_products = products.copy()
_clients  = clients.copy()

# ---------- Asegurar tipos de fecha ----------
_orders['order_timestamp'] = pd.to_datetime(_orders['order_timestamp'], errors='coerce')

# si no existe order_date, lo creamos
if 'order_date' not in _orders.columns:
    _orders['order_date'] = _orders['order_timestamp'].dt.date
_orders['order_date'] = pd.to_datetime(_orders['order_date'], errors='coerce')

_products['last_updated'] = pd.to_datetime(_products['last_updated'], errors='coerce')
_clients['last_seen']     = pd.to_datetime(_clients['last_seen'], errors='coerce')

# ---------- Recorte temporal (máximo 31/12/2024) ----------
_orders   = _orders[_orders['order_timestamp'] <= MAX_DATE]
_products = _products[_products['last_updated'].isna() | (_products['last_updated'] <= MAX_DATE)]

# ---------- Unir orders con products para traer category / last_updated ----------
ord_prod = _orders.merge(
    _products[['product_id','category','last_updated']],
    on='product_id', how='left'
)

# ---------- Función modo segura ----------
def _mode_safe(s):
    m = s.mode(dropna=True)
    return m.iloc[0] if not m.empty else pd.NA

# ---------- Agregados a nivel cliente (solo variables pedidas) ----------
agg_orders = ord_prod.groupby('client_id').agg(
    n_orders               = ('order_id', 'count'),
    sum_order_price        = ('order_price', 'sum'),
    avg_order_price        = ('order_price', 'mean'),
    last_order_timestamp   = ('order_timestamp', 'max'),
    last_order_date        = ('order_date', 'max'),
    sum_shipping_cost      = ('shipping_cost', 'sum'),
    avg_shipping_cost      = ('shipping_cost', 'mean'),
    n_unique_product_cat   = ('product_category', 'nunique'),
    top_product_category   = ('product_category', _mode_safe),
    # desde products:
    n_unique_prod_category = ('category', 'nunique'),
    top_prod_category      = ('category', _mode_safe),
    max_prod_last_updated  = ('last_updated', 'max')
).reset_index()

# ---------- Base de clientes ----------
base_clients = _clients[['client_id','gender','age','last_seen']].copy()

# ---------- Dataset final por cliente ----------
dataset = base_clients.merge(agg_orders, on='client_id', how='left')

# Recencia al corte y churn (90 días)
dataset['recency_days'] = (CUTOFF - dataset['last_order_timestamp']).dt.days
dataset['churn'] = (
    dataset['last_order_timestamp'].isna() |
    (dataset['last_order_timestamp'] <= (CUTOFF - pd.Timedelta(days=HORIZON)))
).astype(int)

# Completar nulos razonables
fill_map = {
    'n_orders': 0, 'sum_order_price': 0.0, 'avg_order_price': 0.0,
    'sum_shipping_cost': 0.0, 'avg_shipping_cost': 0.0,
    'n_unique_product_cat': 0, 'n_unique_prod_category': 0,
    'recency_days': 10_000
}
dataset = dataset.fillna(fill_map)

# Orden sugerido de columnas
cols = [
    'client_id','gender','age','last_seen',
    'n_orders','sum_order_price','avg_order_price',
    'sum_shipping_cost','avg_shipping_cost',
    'last_order_timestamp','last_order_date','recency_days',
    'n_unique_product_cat','top_product_category',
    'n_unique_prod_category','top_prod_category','max_prod_last_updated',
    'churn'
]
dataset = dataset[cols]

print(dataset.shape)
dataset.head(10)


(2357, 18)


,client_id,gender,age,last_seen,n_orders,sum_order_price,avg_order_price,sum_shipping_cost,avg_shipping_cost,last_order_timestamp,last_order_date,recency_days,n_unique_product_cat,top_product_category,n_unique_prod_category,top_prod_category,max_prod_last_updated,churn
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,M,42,2025-06-19,44.0,2306.18,52.413182,224.58,5.104091,2024-12-16 21:01:32,2024-12-16,-108.0,5.0,Household,0.0,<NA>,NaT,0
1,36e48bdd-db11-4abe-9526-cfc90e68924d,M,58,2023-12-09,122.0,7236.93,59.319098,664.55,5.447131,2024-12-26 10:58:03,2024-12-26,-118.0,5.0,Personal Care,0.0,<NA>,NaT,0
2,145c22df-3579-412e-bc12-b4fce70abaf3,M,28,2025-05-03,37.0,2076.58,56.123784,190.14,5.138919,2024-12-29 22:46:06,2024-12-29,-121.0,5.0,Technology,0.0,<NA>,NaT,0
3,90c4a925-e51f-4dac-9193-2d9aec97a472,M,19,2025-08-11,13.0,614.39,47.260769,53.67,4.128462,2024-11-19 02:28:56,2024-11-19,-81.0,3.0,Technology,0.0,<NA>,NaT,0
4,853f711a-4c36-40b4-b5d0-6207152cd793,M,36,2025-03-21,0.0,0.00,0.000000,0.00,0.000000,NaT,NaT,10000.0,0.0,NaN,0.0,NaN,NaT,1
5,22cdc8c6-e370-4fb5-aa37-91f1d7a6fba6,M,25,2025-08-08,32.0,1876.15,58.629688,152.84,4.776250,2024-12-01 14:00:05,2024-12-01,-93.0,5.0,Technology,0.0,<NA>,NaT,0
6,b4f357f5-3b86-4aae-927f-0ee4da25baf9,F,47,2025-05-27,59.0,2732.66,46.316271,271.01,4.593390,2024-12-25 06:17:58,2024-12-25,-117.0,5.0,Personal Care,0.0,<NA>,NaT,0
7,f05a7bb5-a3c2-4ee7-a5a1-2aab5f885183,M,46,2024-04-15,15.0,808.03,53.868667,83.08,5.538667,2024-12-30 02:55:39,2024-12-30,-122.0,5.0,Household,0.0,<NA>,NaT,0
8,e66b3d22-5a63-43b4-86e3-df61f8e3c415,M,28,2025-08-30,18.0,998.86,55.492222,107.13,5.951667,2024-12-12 00:55:33,2024-12-12,-104.0,3.0,Technology,0.0,<NA>,NaT,0
9,f2f46b96-3a9e-4b8e-8efc-d63c724256d9,M,24,2025-05-24,61.0,2982.67,48.896230,293.67,4.814262,2024-12-26 17:20:06,2024-12-26,-118.0,5.0,Technology,0.0,<NA>,NaT,0
